### Library Import

In [28]:
import pandas as pd

### Generate Trip and Network 

In [29]:
def write_tntp_files(
    od_matrix_path,
    link_csv_path,
    node_csv_path,
    network_name,
    threshold,
    demand_reduction_factor,
    flooded_factor,
    partially_flooded_factor,
    output_folder="./results/"
):
    """
    Generates trip and network files in TNTP format with demand and capacity adjustments.

    Parameters:
    - od_matrix_path (str): Path to the OD matrix CSV file.
    - link_csv_path (str): Path to the links CSV file.
    - node_csv_path (str): Path to the nodes CSV file.
    - network_name (str): Name of the network (used for output file naming).
    - demand_reduction_factor (float): Factor to scale the total demand.
    - flooded_factor (float): Factor to scale capacity for flooded links.
    - partially_flooded_factor (float): Factor to scale capacity for partially flooded links.
    - output_folder (str): Path to the folder for saving output files.
    """
    # === Step 1: Trip Generator ===
    # Load OD matrix
    od_data = pd.read_csv(od_matrix_path)
    od_matrix = od_data.pivot(index='d_zone_new_id', columns='o_zone_new_id', values='volume').fillna(0)

    # Scale demand
    total_demand = od_matrix.sum().sum()
    scaled_od_matrix = od_matrix * demand_reduction_factor
    scaled_total_demand = scaled_od_matrix.sum().sum()

    # Write trip file
    trip_file_name = f"{output_folder}{network_name}_TR{threshold}_DR{demand_reduction_factor}_FF{flooded_factor}_PF{partially_flooded_factor}_trips.txt"
    with open(trip_file_name, 'w') as f:
        # Metadata
        f.write("<NUMBER OF ZONES> {}\n".format(len(od_matrix)))
        f.write("<TOTAL OD FLOW> {:.1f}\n".format(scaled_total_demand))
        f.write("<END OF METADATA>\n\n\n")

        # Write OD pairs
        for origin in scaled_od_matrix.index:
            f.write("Origin  {}\n".format(origin))
            lines = []
            for destination, flow in scaled_od_matrix.loc[origin].items():
                line = "  {:>4} : {:>10.1f};".format(destination, flow)
                lines.append(line)
            for i in range(0, len(lines), 5):
                f.write(' '.join(lines[i:i+5]) + '\n')
            f.write('\n')
    print(f"Trip file created: {trip_file_name}")

    # === Step 2: Network Generator ===
    # Load link and node data
    df_links = pd.read_csv(link_csv_path, low_memory=False)
    df_nodes = pd.read_csv(node_csv_path, low_memory=False)

    # Clean and adjust link data
    df_links.columns = df_links.columns.str.strip()
    #print(df_links.__dict__)
    
    df_links['from_node_id'] = df_links['from_node_id'].astype(int)
    df_links['to_node_id'] = df_links['to_node_id'].astype(int)
    df_links['B'] = 0.15
    df_links['Power'] = 4
    df_links['Type'] = 1
    df_links[';'] = ';'
    df_links['~'] = '        '
           
    # Rename columns to match TNTP format
    df_links = df_links.rename(columns={
        'from_node_id': 'Init node',
        'to_node_id': 'Term node',
        'capacity': 'Capacity',
        'length': 'Length',
        'travel_time': 'Free Flow Time',
        'Speed Limit': 'Speed limit',
        'width': 'Width',
        'flood_risk': 'Flood',
        'elevation_from': 'el_from',
        'elevation_to': 'el_to'
    })

    # Adjust capacity based on flood risk
    def adjust_capacity(row):
        if row['Flood'] == 'flooded':
            return row['Capacity'] * flooded_factor
        elif row['Flood'] == 'partially-flooded':
            return row['Capacity'] * partially_flooded_factor
        else:  # 'safe'
            return row['Capacity']

    df_links['Capacity'] = df_links.apply(adjust_capacity, axis=1)

    # Sort and prepare data for TNTP
    df_links.sort_values(by=['Init node'], inplace=True)
    tntp_data_with_constants = df_links[[
        '~', 'Init node', 'Term node', 'Capacity', 'Length', 'Free Flow Time',
        'B', 'Power', 'Speed limit', 'Width', 'Type', 'Flood', 'el_from', 'el_to', ';'
    ]]

    # Metadata
    number_of_zones = len(od_matrix)
    number_of_nodes = len(df_nodes)
    number_of_links = len(df_links)
    first_thru_node = 1
    metadata = f"<NUMBER OF ZONES> {number_of_zones}\t\t\t\t\t\n" \
               f"<NUMBER OF NODES> {number_of_nodes}\t\t\t\t\t\n" \
               f"<FIRST THRU NODE> {first_thru_node}\t\t\t\t\t\n" \
               f"<NUMBER OF LINKS> {number_of_links}\t\t\t\t\t\n" \
               f"<END OF METADATA>\n\n\n"

    # Write network file
    network_file_name = f"{output_folder}{network_name}_TR{threshold}_DR{demand_reduction_factor}_FF{flooded_factor}_PF{partially_flooded_factor}_net.txt"
    with open(network_file_name, 'w') as f:
        f.write(metadata)
        tntp_data_with_constants.to_csv(f, sep='\t', index=False, header=True, mode='a')
    print(f"Network file created: {network_file_name}")


# Input your information here
import os

TR = [0.2,1,2]
DR = [0.4,0.75,1]
FF = [0.5,0.6]
PF = [0.7,0.8]

folder_name = 'C:/Users/tasni/Downloads/tap-b-master (1)/tap-b-master/Parameters_Hyde_County/'
if not os.path.exists(folder_name):
	os.makedirs(folder_name)

for ix in range(0,len(TR)):
    for jx in range(0,len(DR)):
        for kx in range(0,len(FF)):
            for lx in range(0,len(PF)):
                sp = 'HD_TR'+str(TR[ix])+'_DR'+str(DR[jx])+'_FF'+str(FF[kx])+'_PF'+str(PF[lx])
                print(sp)
                write_tntp_files(
                    od_matrix_path='./data/'+str(TR[ix])+'m/demand_modified.csv',
                    link_csv_path='./data/'+str(TR[ix])+'m/link.csv',
                    node_csv_path='./data/'+str(TR[ix])+'m/node.csv',
                    threshold =str(TR[ix]),
                    network_name='HD',
                    demand_reduction_factor=DR[jx],
                    flooded_factor=FF[kx],
                    partially_flooded_factor=PF[lx],
                    output_folder=folder_name
                )
                print('\n\n')

HD_TR0.2_DR0.4_FF0.5_PF0.7
Trip file created: C:/Users/tasni/Downloads/tap-b-master (1)/tap-b-master/Parameters_Hyde_County/HD_TR0.2_DR0.4_FF0.5_PF0.7_trips.txt
Network file created: C:/Users/tasni/Downloads/tap-b-master (1)/tap-b-master/Parameters_Hyde_County/HD_TR0.2_DR0.4_FF0.5_PF0.7_net.txt



HD_TR0.2_DR0.4_FF0.5_PF0.8
Trip file created: C:/Users/tasni/Downloads/tap-b-master (1)/tap-b-master/Parameters_Hyde_County/HD_TR0.2_DR0.4_FF0.5_PF0.8_trips.txt
Network file created: C:/Users/tasni/Downloads/tap-b-master (1)/tap-b-master/Parameters_Hyde_County/HD_TR0.2_DR0.4_FF0.5_PF0.8_net.txt



HD_TR0.2_DR0.4_FF0.6_PF0.7
Trip file created: C:/Users/tasni/Downloads/tap-b-master (1)/tap-b-master/Parameters_Hyde_County/HD_TR0.2_DR0.4_FF0.6_PF0.7_trips.txt
Network file created: C:/Users/tasni/Downloads/tap-b-master (1)/tap-b-master/Parameters_Hyde_County/HD_TR0.2_DR0.4_FF0.6_PF0.7_net.txt



HD_TR0.2_DR0.4_FF0.6_PF0.8
Trip file created: C:/Users/tasni/Downloads/tap-b-master (1)/tap-b-master/Pa